In [2]:
import cv2
import os
import math
import random
import glob
import sys
import numpy as np 
from PIL import Image

In [3]:
image_path = "image/origin.jpeg"
# mat -> [col, row]
cols, rows = 2, 2

In [4]:
# 분할된 이미지를 확인하기 위한 코드
import matplotlib.pyplot as plt

def show_img(img_set):
    fig = plt.figure()
    idx = 1
    for i in range(len(img_set)):
        ax = fig.add_subplot(rows, cols, idx)
        ax.imshow(cv2.cvtColor(img_set[i], cv2.COLOR_BGR2RGB))
        ax.set_xticks([])
        ax.set_yticks([])
        idx +=1
    plt.show()


In [16]:
# 이미지 로드
img = cv2.imread(image_path)
# 이미지 해상도 확인 
h, w = img.shape[:2]
#print(h,w)
# 이미지 분할 -> 입력받은 매트릭 개로 분할
# 가로, 세로 크기 계산
crop_h = (h/rows if (h/rows).is_integer() else math.trunc(h/rows))
crop_w = (w/cols if (w/cols).is_integer() else math.trunc(w/cols))
#print(crop_h, crop_w)

# col by row 이미지 자르기
cropped_images = []
ratio = 0.3
for i in range(rows):
    for j in range(cols):
        left = int(j * crop_w)
        top = int(i * crop_h)
        right = int((j + 1) * crop_w)
        bottom = int((i + 1) * crop_h)
        print(f"left:{left}, top:{top}, right:{right}, bottom:{bottom}")
        #cropped_image = img[int(top - math.trunc(top * 0.3)):int(bottom + math.trunc(bottom * 0.3)), int(left - math.trunc(left * 0.3)):int(right + math.trunc(right * 0.3))]
        cropped_image = img[int(top):int(bottom), int(left):int(right)]
        # print(int(top - top * 0.3), int(bottom + bottom * 0.3), int(left - left * 0.3) , int(right + right * 0.3))

# 분할된 이미가 각 50%의 확률로 미러링, 플립핑, 90도 변환이 되록하여 아웃풋 이미지들을 만들어 낸뒤 저장
        '''
        # mirroring (좌우)
        if random.random() > 0.5:
            cropped_image = cv2.flip(cropped_image, 1)
            #print("mirroring is done")
        # flipping (상하)
        if random.random() > 0.5:
            cropped_image = cv2.flip(cropped_image, 0)
            #print("flipping is done")
        # 90도변환
        if random.random() > 0.5:
            cropped_image = cv2.rotate(cropped_image, cv2.ROTATE_90_CLOCKWISE)
            #print("rotatind is done")
        '''
        cropped_images.append(cropped_image)

# 이미지 저장 -> 정보를 알 수 없게 
if not os.path.exists("image/crooped_images/"):
    os.mkdir("image/crooped_images/")
else:
    [os.remove(f) for f in glob.glob("image/crooped_images/*.jpg")]
    
for img in cropped_images:
    cv2.imwrite(f"image/crooped_images/cropped_{int(round(random.random(),7) * 10000000)}.jpg", img)


left:0, top:0, right:384, bottom:512
left:384, top:0, right:768, bottom:512
left:0, top:512, right:384, bottom:1024
left:384, top:512, right:768, bottom:1024


In [17]:
show_img(cropped_images)

In [7]:
stitcher = cv2.Stitcher_create()

def stitching_img(stan_img, proc_img):
    
    ret, dst = stitcher.stitch([stan_img, proc_img])

    if ret != cv2.Stitcher_OK:
        print('Stitch failed')
        return None, False
        
    return dst, True   
    

In [ ]:


# gray sclae & down resolution
processing_img = [cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (0,0), fx=4, fy=4, interpolation = cv2.INTER_AREA) for img in cropped_images]

stand_img = processing_img[0]
proc_img = processing_img[1]

# 1. not transformation
results, flag = stitching_img(stand_img, proc_img)
if flag == True:
    plt.imshow(results)

In [ ]:
# 2. mirroring
result, flag = stitching_img(stand_img, cv2.flip(proc_img, 1))
if result != None:
    plt.imshow(result)
print(flag)

In [ ]:
# 3. flipping 
result, flag = stitching_img(stand_img, cv2.flip(proc_img, 0))
if result != None:
    plt.imshow(result)
print(flag)

In [ ]:
# 4. 90 degree rotation
result, flag = stitching_img(stand_img, cv2.rotate(proc_img, cv2.ROTATE_90_COUNTERCLOCKWISE))
if result != None:
    plt.imshow(result)
print(flag)

In [ ]:
# 5. mirroring & filpping
result, flag = stitching_img(stand_img, cv2.flip(cv2.flip(proc_img, 1), 0))
if result != None:
    plt.imshow(result)
print(flag)

In [ ]:
# 6. mirroring & 90 degree
result, flag = stitching_img(stand_img, cv2.rotate(cv2.flip(proc_img,1), cv2.ROTATE_90_COUNTERCLOCKWISE))
if result != None:
    plt.imshow(result)
print(flag)

In [ ]:
# 7. flipping & 90 degree
result, flag = stitching_img(stand_img, cv2.rotate(cv2.flip(proc_img, 0), cv2.ROTATE_90_COUNTERCLOCKWISE))
if result != None:
    plt.imshow(result)
print(flag)

In [ ]:
# 8. mirroring & flipping & 90 degree
result, flag = stitching_img(stand_img, cv2.rotate(cv2.flip(cv2.flip(proc_img, 1), 0), cv2.ROTATE_90_COUNTERCLOCKWISE))
if result != None:
    plt.imshow(result)
print(flag)

In [19]:
import cv2
import os
import math
import random
import glob
import sys
import numpy as np
from PIL import Image

image_path = "image/origin.jpeg"
# mat -> [col, row]
cols, rows = 2, 2
stitcher = cv2.Stitcher_create()


def stitching_img(stan_img, proc_img):

    ret, dst = stitcher.stitch([stan_img, proc_img])

    if ret != cv2.Stitcher_OK:
        print('Stitch failed')
        return None, False

    return dst, True

def set_resolution(cropped_images):
    res_set = []

    for idx, img in enumerate(cropped_images):
        if idx == 0:
            stan_img_shape = img.shape[:2]
            print(stan_img_shape)
            res_set.append(img)
        else:
            if stan_img_shape == img.shape[:2]:
                #print("ok")
                res_set.append(img)
            else:
                sub_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
                #print(sub_img.shape[:2])
                res_set.append(sub_img)

    return res_set

def show_img(img_list):
    for idx, img in enumerate(img_list):
        cv2.imshow(f"img_{idx}", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [30]:
cropped_images  = [cv2.imread(f) for f in glob.glob("image/crooped_images/*.jpg")]

# Gray Scale 변환
processed_img = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in cropped_images]

# Gaussian Blur
processed_img = [cv2.GaussianBlur(img, (0,0), 2) for img in processed_img]

res_imgs = set_resolution(processed_img)

resolutions = [i.shape[:2] for i in res_imgs]
print(resolutions)

# 2. image merge(using stitching)

#show_img(processed_img)
canny_img = [cv2.Canny(img, 60,200) for img in processed_img]

show_img(res_imgs)

(512, 384)
[(512, 384), (512, 384), (512, 384), (512, 384)]


In [31]:
show_img(res_imgs)

In [40]:
img1 = res_imgs[0]
img2 = res_imgs[2]
v_img = cv2.hconcat([img1, img2])
v_c_img = cv2.hconcat([cv2.Canny(img1, 60,200), cv2.Canny(img2, 60,200)])
v_canny = cv2.Canny(v_img, 60,200)
show_img([v_c_img, v_canny, (v_canny - v_c_img)])

print(np.sum(v_canny - v_c_img))


28052
